# A

In [6]:
import pandas as pd
import sqlite3
import requests
import json

# Load CSV data
df = pd.read_csv('Mobiles_Dataset.csv')
TABLE_NAME = "mobiles"
# Create SQLite database
conn = sqlite3.connect('my_test_sql.db')
df.to_sql(TABLE_NAME, conn, if_exists='replace', index=False)


# Get table structure
cursor = conn.cursor()
cursor.execute(f"PRAGMA table_info({TABLE_NAME})")
columns = [column[1] for column in cursor.fetchall()]
MODEL_NAME = "smollm:1.7b"
MODEL_NAME = "gemma2:2b"
# MODEL_NAME = "phi3.5"
# Function to generate SQL query using Ollama
def generate_sql_query(user_query, table_name, columns):
    prompt = f"""
    Table name: {table_name}
    Columns: {', '.join(columns)}
    User query: {user_query}

    Based on the table structure and the user's query, generate a SQL query that answers the user's question.
    Only use the columns that are provided. If the query cannot be answered with the given columns, explain why.
    
    SQL Query:
    """
    
    response = requests.post('http://localhost:11434/api/generate', 
                             json={
                                 "model": MODEL_NAME,
                                 "prompt": prompt
                             })
    if response.status_code == 200:
        print(response)
        return response.json()['response'].strip()
    else:
        raise Exception(f"Error generating query: {response.text}")

# Function to execute SQL query and get results
def execute_sql_query(query):
    try:
        result = pd.read_sql_query(query, conn)
        return result.to_dict(orient='records')
    except Exception as e:
        return f"Error executing query: {str(e)}"

# Function to generate answer based on SQL results
def generate_answer(user_query, sql_query, sql_results):
    prompt = f"""
    User query: {user_query}
    SQL Query: {sql_query}
    SQL Results: {json.dumps(sql_results, indent=2)}

    Based on the user's query and the SQL results, provide a concise answer to the user's question.
    If the SQL query didn't provide enough information, explain why.

    Answer:
    """
    
    response = requests.post('http://localhost:11434/api/generate', 
                             json={
                                 "model": MODEL_NAME,
                                 "prompt": prompt
                             })
    if response.status_code == 200:
        return response.json()['response'].strip()
    else:
        raise Exception(f"Error generating answer: {response.text}")

# Main function to process user query
def process_query(user_query):
    sql_query = generate_sql_query(user_query, TABLE_NAME, columns)
    sql_results = execute_sql_query(sql_query)
    answer = generate_answer(user_query, sql_query, sql_results)
    return answer
    return {
        "user_query": user_query,
        "sql_query": sql_query,
        "sql_results": sql_results,
        "answer": answer
    }

# Example usage


In [7]:
user_query = "Give me list of all the phone"
result = process_query(user_query)
print(json.dumps(result, indent=2))

<Response [200]>


JSONDecodeError: Extra data: line 2 column 1 (char 96)

In [12]:
def generate_sql_query(user_query, table_name, columns):
    prompt = f"""
    Table name: {table_name}
    Columns: {', '.join(columns)}
    User query: {user_query}

    Based on the table structure and the user's query, generate a SQL query that answers the user's question.
    Only use the columns that are provided. If the query cannot be answered with the given columns, explain why.
    
    SQL Query:
    """
    
    response = requests.post('http://localhost:11434/api/generate', 
                             json={
                                 "model": "smollm:1.7b",
                                 "prompt": prompt
                             },
                             stream=True)
    
    if response.status_code == 200:
        full_response = ""
        for line in response.iter_lines():
            if line:
                try:
                    json_response = json.loads(line)
                    if 'response' in json_response:
                        full_response += json_response['response']
                except json.JSONDecodeError:
                    print(f"Failed to decode JSON: {line}")
        return full_response.strip()
    else:
        raise Exception(f"Error generating query: {response.text}")

In [18]:
import pandas as pd
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Load CSV data
df = pd.read_csv('Mobiles_Dataset.csv')
MODEL_NAME = "gemma2:2b"
# Initialize Ollama LLM
llm = Ollama(
    model=MODEL_NAME,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# Create pandas DataFrame agent
agent = create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code=True, handle_parsing_errors=True)

def process_query(user_query):
    try:
        response = agent.run(user_query)
        return {
            "user_query": user_query,
            "answer": response
        }
    except Exception as e:
        return {
            "user_query": user_query,
            "error": str(e)
        }

# Example usage
user_query = "list all the iphones"
result = process_query(user_query)
print(result)

C:\Users\rajab\miniconda3\envs\py3_120\Lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...
There are no iPhones listed in this data. 

The data provided shows OnePlus N20 SE options with different storage sizes (64GB and 128GB) in various colors. 


Let me know if you have any other questions! {'user_query': 'list all the iphones', 'error': 'An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `There are no iPhones listed in this data. \n\nThe data provided shows OnePlus N20 SE options with different storage sizes (64GB and 128GB) in various colors. \n\n\nLet me know if you have any other questions! `'}


In [14]:
!start

# B

In [14]:
import pandas as pd
import sqlite3
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.schema import Document
from langchain.chains import RetrievalQA, LLMChain
from langchain.llms import Ollama
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.agents import AgentType
from langchain.prompts import PromptTemplate

def create_or_load_vectorstore(documents, index_name="faiss_index"):
    embeddings = HuggingFaceEmbeddings()
    
    if documents:
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        texts = text_splitter.split_documents(documents)
        vectorstore = FAISS.from_documents(texts, embeddings)
        vectorstore.save_local(index_name)
    else:
        vectorstore = FAISS.load_local(index_name, embeddings)
    
    return vectorstore

def setup_llm():
    return Ollama(model="gemma2:2b")

def setup_retriever(vectorstore):
    return vectorstore.as_retriever()

def setup_pandas_agent(llm, df):
    if df is not None:
        return create_pandas_dataframe_agent(
            llm,
            df,
            verbose=True,
            agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,allow_dangerous_code=True
        )
    return None

def setup_sql_query_chain(llm):
    sql_query_prompt = PromptTemplate(
        input_variables=["question", "table_info"],
        template="Given the following SQL table information:\n{table_info}\n\nGenerate a SQL query to answer the following question: {question}"
    )
    return LLMChain(llm=llm, prompt=sql_query_prompt)

def get_table_info(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    table_info = []
    for table in tables:
        cursor.execute(f"PRAGMA table_info({table[0]});")
        columns = cursor.fetchall()
        table_info.append(f"Table: {table[0]}\nColumns: {', '.join([col[1] for col in columns])}")
    return "\n\n".join(table_info)

def execute_sql_query(conn, query):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()
    except sqlite3.Error as e:
        return f"SQLite error: {e}"

class CombinedChain:
    def __init__(self, retrieval_qa, pandas_agent, db_connection, sql_query_chain):
        self.retrieval_qa = retrieval_qa
        self.pandas_agent = pandas_agent
        self.db_connection = db_connection
        self.sql_query_chain = sql_query_chain
    
    def get_rag_answer(self, query):
        if self.retrieval_qa:
            rag_result = self.retrieval_qa({"query": query})
            return rag_result['result'], rag_result['source_documents']
        return "No RAG system available", []
    
    def get_pandas_answer(self, query):
        if self.pandas_agent:
            return self.pandas_agent.run(query)
        return "No pandas agent available"
    
    def get_sql_answer(self, query):
        if self.db_connection:
            table_info = get_table_info(self.db_connection)
            sql_query = self.sql_query_chain.run(question=query, table_info=table_info)
            return execute_sql_query(self.db_connection, sql_query)
        return "No database connection available"
    
    def __call__(self, query):
        rag_answer, source_documents = self.get_rag_answer(query)
        pandas_result = self.get_pandas_answer(query)
        sql_result = self.get_sql_answer(query)
        
        return {
            "rag_answer": rag_answer,
            "pandas_result": pandas_result,
            "sql_result": sql_result,
            "source_documents": source_documents
        }

def create_rag_system(text_documents=None, csv_path=None, db_path=None):
    llm = setup_llm()
    
    # Set up RAG system for text documents
    retrieval_qa = None
    if text_documents:
        vectorstore = create_or_load_vectorstore(text_documents)
        retriever = setup_retriever(vectorstore)
        retrieval_qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True
        )
    
    # Set up pandas agent for CSV
    df = pd.read_csv(csv_path) if csv_path else None
    pandas_agent = setup_pandas_agent(llm, df)
    
    # Set up SQL connection and query chain
    conn = sqlite3.connect(db_path) if db_path else None
    sql_query_chain = setup_sql_query_chain(llm)
    
    return CombinedChain(retrieval_qa, pandas_agent, conn, sql_query_chain)

def add_new_text_documents(file_paths, index_name="faiss_index"):
    new_documents = []
    for file_path in file_paths:
        new_documents.extend(TextLoader(file_path).load())
    
    create_or_load_vectorstore(new_documents, index_name)

def ask_question(query: str, rag_system):
    result = rag_system(query)
    return {
        "rag_answer": result['rag_answer'],
        "pandas_result": result['pandas_result'],
        "sql_result": result['sql_result'],
        "source_documents": [doc.page_content for doc in result['source_documents']] if result['source_documents'] else []
    }

# Example usage
# rag_system = create_rag_system(text_documents, csv_path="Mobiles_Dataset.csv")

# Add new text documents if needed
# new_file_paths = ["new_text_document1.txt", "new_text_document2.txt"]
# add_new_text_documents(new_file_paths)

# Use the updated system
updated_rag_system = create_rag_system(csv_path="Mobiles_Dataset.csv", db_path="your_database.sqlite")



In [15]:
response = ask_question("How many unique phones are there?", updated_rag_system)
print("RAG Answer:", response['rag_answer'])
print("Pandas Result:", response['pandas_result'])
print("SQL Result:", response['sql_result'])
print("Source Documents:", response['source_documents'])



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `There are **3** unique phones listed in the provided data. `

In [16]:
!start